In [1]:
import numpy as np

In [7]:
t = np.ones(8)
print(np.diag(t))

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


In [6]:
state = np.asarray([[-2, -3, -4, -5, -6, -4, -3, -2],
                   [-1, -1, -1, -1, -1, -1, -1, -1],
                   [0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0],
                   [1, 1, 1, 1, 1, 1, 1, 1],
                   [2, 3, 4, 5, 6, 4, 3, 2]])


(8, 8)


In [43]:
%%time
# TODO: En Passant - Requires knowing the previous board state.
x, y = np.where(state==1)
x = x.reshape(len(x), 1)
y = y.reshape(len(y), 1)

pawns = np.append(x,y,axis=1)

s = []
for pos in pawns:
    s1 = np.copy(state)
    
    # Ignore finding moves for this pawn if there is a piece in front of it.
    if not s1[pos[0]-1, pos[1]] == 0:
        continue
    
    # Moves 1
    s1[pos[0], pos[1]] = 0
    s1[pos[0]-1, pos[1]] = 1
    s.append(np.copy(s1))
    
    # Moves 2
    if pos[0] == 6:
        s1[pos[0]-1, pos[1]] = 0
        s1[pos[0]-2, pos[1]] = 1
        s.append(np.copy(s1))

print(s[0])

[[-2 -3 -4 -5 -6 -4 -3 -2]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0]
 [ 0  1  1  1  1  1  1  1]
 [ 2  3  4  5  6  4  3  2]]
CPU times: user 808 µs, sys: 298 µs, total: 1.11 ms
Wall time: 927 µs


In [299]:
import xml.etree.ElementTree as ET
from IPython.display import SVG

def display_board(board_state):
    # Parses the board in first as a background.
    tree = ET.ElementTree()
    tree.parse("pieces/board.svg")
    board = tree.getroot()

    # Dict containing a conversion between the piece number and the piece name.
    piece_names = {1:"wp", 2:"wr", 3:"wkn", 4:"wb", 5:"wq", 6:"wk",
                   -1:"bp", -2:"br", -3:"bkn", -4:"bb", -5:"bq", -6:"bk"}

    # Numpy iterator that gives us the position as well as the piece number
    it = np.nditer(board_state, flags=['multi_index'])

    while not it.finished:
        pos = it.multi_index
        piece = it[0]

        # 0 is an empty space so we increment the iterator and skip the rest.
        if piece == 0:
            it.iternext()
            continue

        # Parses the raw piece svg file into XML
        tree.parse("pieces/" + piece_names[int(piece)] + ".svg")
        piece = tree.getroot()

        # The positioning of the piece. Each square is 50x50 pixels.
        x_pos = pos[1] * 50
        y_pos = pos[0] * 50
        piece.set("x", str(x_pos))
        piece.set("y", str(y_pos))
        
        # Adds the piece as a subelement of the board.
        board.append(piece)

        it.iternext()

    return SVG(ET.tostring(board))

In [300]:
display_board(state)